### Purged K-Fold Cross Validation

정보 누출을 감소시키는 방법 중 하나는 훈련 데이터셋에서 현재 테스트세트와 레이블이 중첩된 모든 관측값을 제거하는 것이다. Prado 교수는 이 절차를 Purging이라고 불렀다. 게다가 금융데이터는 대부분 시계열적이므로 자기상관을 보이는 시계열을 종종 포함한다. 테스트셋에 있는 관측값을 즉시 따르는 훈련셋 관측값을 제거해야 한다. 저자인 Prado교수는 이것을 Embargo process라고 부른다

#### Purging the Training Set

label이 $Y_j$인 테스트 관측값이 정보 집합 $\Phi_j$에 근거해 결정됐다고 가정해 보자. 정보의 누출 형태를 방지하고자 정보 집합 $\Phi_i$에 근거해서 label $Y_i$를 결정한 모든 관측값을 훈련 데이터에서 제거해서 $\Phi_i \cap \Phi_j = \emptyset$이 되게 한다. 즉, 테스트셋에 정보의 Concurrency가 존재하는 데이터를 훈련 데이터셋에서 삭제하는 과정이다.

특별히 $Y_i$와 $Y_j$가 공존할 때에는 언제나 양쪽 label이 적어도 하나의 공통 무작위 추출에 달려 있다는 관점에서 두 관측값 $i$와 $j$ 사이에 정보의 중첩이 있다고 결정한다. 예를 들어서, 닫힌 구간 $t \in [t_{j,0}, t_{j,1}], Y_j = f\left[ [t_{j,0}, t_{j,1}]\right]$에서 관측값의 함수인 label $Y_j$를 고려해 보자. 예를 들어서, triple barrier labeling의 관점에서는 label이 인덱스가 $t_{j,0}$와 $t_{j,1}$ 사이의 가격 바 수익률의 부호, 즉 $\mathrm{sgn}[r_{t_{j, 0}, t_{j, 1}}]$이다. label $Y_i = f\left[ [t_{i,0}, t_{i,1}]\right]$은 다음 세 가지 충분 조건 중 하나가 만족되면 $Y_j$와 중첩된다

1. $t_{j,0} \leq t_{i,0} \leq t_{j,1}$
2. $t_{j,0} \leq t_{i,1} \leq t_{j,1}$
3. $t_{i,0} \leq t_{j,0} \leq t_{j,1} \leq t_{i,1}$

아래의 코드는 훈련 데이터셋에서 관측값 제거를 구현한다. 처음과 마지막 테스트 관측값 사이에서는 훈련 관측값이 발생하지 않는다는 관점에서 테스트셋이 연속이면 제거는 가속화될 수 있다.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

start_date = '2000-01-01'
end_date = '2024-05-11'
ticker = 'AAPL'
data = yf.download(
    ticker, 
    start = start_date, 
    end = end_date
)

triple_barrier_events = pd.read_parquet('../../../Data/triple_barrier_events.parquet')

[*********************100%%**********************]  1 of 1 completed


In [7]:
from FinancialMachineLearning.cross_validation.cross_validation import get_train_times

def get_embargo_times(times, pct_embargo):
    step = int(times.shape[0] * pct_embargo)
    if step == 0:
        mbrg = pd.Series(times, index=times)
    else:
        initial = pd.Series(times[step:], index=times[:-step])
        tail = pd.Series([times[-1]] * step, index=times[-step:])
        mbrg = pd.concat([initial, tail])
    return mbrg

test_times = get

Date
2024-05-06   NaT
2024-05-07   NaT
2024-05-08   NaT
2024-05-09   NaT
2024-05-10   NaT
Name: t1, dtype: datetime64[ns]

#### Embargo

제거로도 정보 누출을 방지하지 못하는 경우에는 모든 테스트셋 다음의 훈련 관측값에게 embargo를 설정할 수 있다. 훈련 label $Y_i = f\left[ [t_{i,0}, t_{i,1}]\right]$은 $t_{i,1} < t_{j,0}$에서 테스트 시간 $t_{j,0}$에 있었던 정보를 포함하기 때문에 Embargo는 테스트셋 이전의 훈련 관측값에 대해 조치를 취하는 것이 아니다. 달리 말하면 여기서는 오직 테스트 직후 $t_{j,1} \leq t_{i,0} \leq t_{j,1} + h$에 발생하는 train label $Y_i = f\left[ [t_{i,0}, t_{i,1}]\right]$만 고려하기 때문이다.

이 embargo 기간 $h$는 제거 이전에 $Y_i = f\left[ [t_{j,0}, t_{j,1} + h]\right]$로 설정하면 구현할 수 있다. $k \rightarrow T$로 증가시켜도 테스트 성능이 무한정 향상되지 않는다는 것을 확인 가능한 바와 같이 작은 값 $h \approx 0.01T$는 대개 모든 정보 누출을 방지하기에 충분하다. 아래의 코드는 앞서 설명한 embargo process에 대해서 보여준다.

#### Purged K Fold Class

앞서 레이블이 중첩상태일 때 훈련 테스트 데이터 분할을 어떻게 생성할지 알아보았다. 모델 개발의 특서한 맥락에서 제거와 embargo 개념을 소개했다. 일반적으로 Hyper parameter fitting이나 Backtesting, 성과 평가 등에 관계없이 Train / Test data split을 생성할 때마다 중첩된 훈련 관측값을 제거하고 Embargo 해야 한다. 아래의 코드는 `scikit-learn`의 K Fold 클래스를 확장해 테스트 정보가 훈련셋으로 누출될 가능성을 막는 과정이다